In [1]:
import pandas as pd 

es_futures = pd.read_csv("/Users/kennethzhang/Desktop/VolAuction/data/es_futures_1m_all.csv", parse_dates=True, index_col=0)
spx_spot = pd.read_csv("/Users/kennethzhang/Desktop/VolAuction/data/spx_spot_daily.csv", parse_dates=["date"])
atm_strike_map = pd.read_csv("/Users/kennethzhang/Desktop/VolAuction/data/atm_strike_map.csv", parse_dates=["date"])
auction_quotes = pd.read_parquet("/Users/kennethzhang/Desktop/VolAuction/data/auction_daily_quotes.parquet")

In [2]:
es_futures.index = pd.to_datetime(es_futures.index)
es_futures = es_futures.tz_convert("America/New_York")

In [3]:
es_futures

,Open,High,Low,Close,Volume,Dividends,Stock Splits,date
Datetime,,,,,,,,
2025-03-24 00:00:00-04:00,5754.00,5754.00,5752.75,5753.00,0,0.0,0.0,2025-03-24
2025-03-24 00:01:00-04:00,5753.00,5753.00,5752.75,5752.75,75,0.0,0.0,2025-03-24
2025-03-24 00:02:00-04:00,5753.00,5753.25,5752.75,5753.25,18,0.0,0.0,2025-03-24
2025-03-24 00:03:00-04:00,5753.00,5753.00,5752.50,5752.50,85,0.0,0.0,2025-03-24
2025-03-24 00:04:00-04:00,5752.50,5753.25,5752.50,5753.00,62,0.0,0.0,2025-03-24
...,...,...,...,...,...,...,...,...
2025-03-31 16:55:00-04:00,5645.75,5646.00,5645.00,5645.25,632,0.0,0.0,2025-03-31
2025-03-31 16:56:00-04:00,5645.25,5645.75,5644.25,5644.50,420,0.0,0.0,2025-03-31
2025-03-31 16:57:00-04:00,5644.50,5645.00,5644.00,5644.50,344,0.0,0.0,2025-03-31


In [4]:
spx_spot["date"] = pd.to_datetime(spx_spot["date"], utc=True).dt.tz_convert("America/New_York")

In [5]:
def ensure_et_timezone(df, col="date"):
    df[col] = pd.to_datetime(df[col], errors="coerce") 
    if pd.api.types.is_datetime64tz_dtype(df[col]):
        df[col] = df[col].dt.tz_convert("America/New_York")
    else:
        df[col] = df[col].dt.tz_localize("America/New_York")
    return df

In [6]:
spx_spot = ensure_et_timezone(spx_spot)
atm_strike_map = ensure_et_timezone(atm_strike_map)
auction_quotes = ensure_et_timezone(auction_quotes)

/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_51977/482004672.py:3: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df[col]):
/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_51977/482004672.py:3: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df[col]):
/var/folders/bc/0sv4l9h92yb5pz7_7vm4gh900000gn/T/ipykernel_51977/482004672.py:3: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df[col]):


In [7]:
auction_quotes

,date,strike,call_mid,put_mid,call_bid,call_ask,put_bid,put_ask
0,2025-01-28 00:00:00-05:00,5930000,97.15,97.15,96.8,97.5,0.75,0.85
1,2025-01-29 00:00:00-05:00,5970000,90.40,90.40,90.0,90.8,0.95,1.00
2,2025-01-30 00:00:00-05:00,5990000,72.80,72.80,71.4,74.2,1.05,1.15
3,2025-01-31 00:00:00-05:00,1200000,4894.80,4894.80,4887.7,4901.9,0.00,0.05
4,2025-02-03 00:00:00-05:00,5835000,108.90,108.90,106.3,111.5,1.20,1.25
5,2025-02-04 00:00:00-05:00,5905000,92.95,92.95,90.8,95.1,1.25,1.30
6,2025-02-05 00:00:00-05:00,5945000,81.10,81.10,79.5,82.7,1.00,1.05
7,2025-02-06 00:00:00-05:00,6015000,64.10,64.10,61.1,67.1,0.85,0.95
8,2025-02-07 00:00:00-05:00,1400000,4686.05,4686.05,4678.1,4694.0,0.00,0.05
9,2025-02-10 00:00:00-05:00,5990000,70.45,70.45,70.2,70.7,1.05,1.10


In [8]:
spx_spot

,date,Open,Close,High,Low,Volume
0,2025-01-28 00:00:00-05:00,6026.970215,6067.700195,6074.540039,5994.629883,4473640000
1,2025-01-29 00:00:00-05:00,6057.700195,6039.310059,6062.830078,6012.959961,4115310000
2,2025-01-30 00:00:00-05:00,6050.750000,6071.169922,6086.640137,6027.459961,4634620000
3,2025-01-31 00:00:00-05:00,6096.790039,6040.529785,6120.910156,6030.930176,4843770000
4,2025-02-03 00:00:00-05:00,5969.649902,5994.569824,6022.129883,5923.930176,4797210000
5,2025-02-04 00:00:00-05:00,5998.140137,6037.879883,6042.479980,5990.870117,4410160000
6,2025-02-05 00:00:00-05:00,6020.450195,6061.479980,6062.859863,6007.060059,4756250000
7,2025-02-06 00:00:00-05:00,6072.220215,6083.569824,6084.029785,6046.830078,4847120000
8,2025-02-07 00:00:00-05:00,6083.129883,6025.990234,6101.279785,6019.959961,4766900000
9,2025-02-10 00:00:00-05:00,6046.399902,6066.439941,6073.379883,6044.839844,4458760000


In [11]:
atm_strike_map.to_csv("/Users/kennethzhang/Desktop/VolAuction/data/atm_strike_map.csv")

In [10]:
import numpy as np

overnight_results = []

for auction_day in auction_quotes["date"]:
    auction_day = pd.Timestamp(auction_day)
    t_minus_1 = auction_day - pd.Timedelta(days=1)

    spx_before = spx_spot[spx_spot["date"] < auction_day]
    spx_close_t_minus_1 = spx_before.iloc[-1]["Close"] if not spx_before.empty else None

    t1_str = t_minus_1.strftime("%Y-%m-%d")
    t_str = auction_day.strftime("%Y-%m-%d")
    es_start = pd.Timestamp(f"{t1_str} 18:00", tz="America/New_York")
    es_end = pd.Timestamp(f"{t_str} 09:30", tz="America/New_York")

    es_data = es_futures.loc[(es_futures.index >= es_start) & (es_futures.index <= es_end)]

    log_returns = (es_data["Close"] / es_data["Close"].shift(1)).apply(np.log).dropna()
    realized_std = log_returns.std() * np.sqrt(len(log_returns))  

    overnight_results.append({
        "date": auction_day,
        "spx_close_t_minus_1": spx_close_t_minus_1,
        "realized_std": realized_std,
        "n_ticks": len(log_returns)
    })

overnight_df = pd.DataFrame(overnight_results)